# Boundary Value Problem

$$
\epsilon u'' + (1+\epsilon) u' + u = 0, \quad u(0)=0, \quad u(1)=1
$$

where $\epsilon > 0$ (e.g., $\epsilon = 0.01$).

---

## Standardize the ODE

Divide the equation by $\epsilon$:

$$
u'' + \frac{1+\epsilon}{\epsilon} u' + \frac{1}{\epsilon} u = 0
$$

This is a linear second-order ODE with constant coefficients.

---

## Characteristic equation

The characteristic equation is

$$
\epsilon r^2 + (1+\epsilon) r + 1 = 0 \quad \Rightarrow \quad
r^2 + \frac{1+\epsilon}{\epsilon} r + \frac{1}{\epsilon} = 0
$$

Solve for the roots:

$$
r_{1,2} = \frac{-(1+\epsilon)/\epsilon \pm \sqrt{ \left( \frac{1+\epsilon}{\epsilon} \right)^2 - 4/\epsilon }}{2}
$$

---

## General solution

The general solution is

$$
u(x) = C_1 e^{r_1 x} + C_2 e^{r_2 x}
$$

where $r_1$ and $r_2$ are the characteristic roots.

---

##  Apply boundary conditions

1. At $x=0$, $u(0)=0$:

$$
C_1 + C_2 = 0 \quad \Rightarrow \quad C_2 = -C_1
$$

2. At $x=1$, $u(1)=1$:

$$
C_1 e^{r_1} + C_2 e^{r_2} = C_1 (e^{r_1} - e^{r_2}) = 1 \quad \Rightarrow \quad
C_1 = \frac{1}{e^{r_1} - e^{r_2}}
$$

---

## Exact solution

Finally, the exact solution is

$$
u(x) = \frac{ e^{r_1 x} - e^{r_2 x} }{ e^{r_1} - e^{r_2} }
$$

> Note: For small $\epsilon$ (e.g., $\epsilon = 0.01$), the solution exhibits a **boundary layer** near $x=0$ or $x=1$.


In [3]:
import numpy as np

epsilon = 0.01

# ================================================================
# Exact solution
# ================================================================
def exact_solution(x, epsilon):
    r1 = (-(1+epsilon)/epsilon + np.sqrt((1+epsilon)**2/epsilon**2 - 4/epsilon))/2
    r2 = (-(1+epsilon)/epsilon - np.sqrt((1+epsilon)**2/epsilon**2 - 4/epsilon))/2
    return (np.exp(r1*x) - np.exp(r2*x)) / (np.exp(r1) - np.exp(r2))

# ================================================================
# 2nd-order finite difference solver
# ================================================================
def solve_bvp_fd(N, epsilon):
    h = 1.0/N
    x = np.linspace(0,1,N+1)
    A = np.zeros((N+1,N+1))
    b = np.zeros(N+1)

    # Interior points
    for i in range(1,N):
        A[i,i-1] = epsilon/h**2 - (1+epsilon)/(2*h)
        A[i,i]   = -2*epsilon/h**2 + 1
        A[i,i+1] = epsilon/h**2 + (1+epsilon)/(2*h)
        b[i] = 0

    # Dirichlet BCs
    A[0,:] = 0
    A[0,0] = 1
    b[0] = 0

    A[N,:] = 0
    A[N,N] = 1
    b[N] = 1

    u = np.linalg.solve(A,b)
    return x, u

# ================================================================
# Error norms
# ================================================================
def error_norms(N):
    x, u_fd = solve_bvp_fd(N, epsilon)
    u_ex = exact_solution(x, epsilon)
    err = np.abs(u_fd - u_ex)
    L_inf = np.max(err)
    L2 = np.sqrt(np.sum(err**2)*(1/N))
    return L_inf, L2

# ================================================================
# Convergence table
# ================================================================
Ns = [20, 40, 80, 160, 320]
results = []
prev_Linf = prev_L2 = None

for N in Ns:
    L_inf, L2 = error_norms(N)
    if prev_Linf is None:
        rate_inf = rate_L2 = np.nan
    else:
        rate_inf = np.log(prev_Linf / L_inf)/np.log(2)
        rate_L2  = np.log(prev_L2 / L2)/np.log(2)
    results.append([N, L_inf, rate_inf, L2, rate_L2])
    prev_Linf, prev_L2 = L_inf, L2

# ================================================================
# Print table
# ================================================================
header = f"{'N':>6} | {'L_inf error':>15} | {'rate':>7} | {'L2 error':>15} | {'rate':>7}"
line = "-"*len(header)
print("\nConvergence Table (FD for singularly perturbed BVP)")
print(line)
print(header)
print(line)
for N,Linf,rinf,L2,r2 in results:
    print(f"{N:6d} | {Linf:15.8e} | {rinf:7.3f} | {L2:15.8e} | {r2:7.3f}")
print(line)



Convergence Table (FD for singularly perturbed BVP)
--------------------------------------------------------------
     N |     L_inf error |    rate |        L2 error |    rate
--------------------------------------------------------------
    20 |  1.25301997e+00 |     nan |  3.13758863e-01 |     nan
    40 |  5.46186543e-01 |   1.198 |  8.64249704e-02 |   1.860
    80 |  1.56540684e-01 |   1.803 |  2.00360805e-02 |   2.109
   160 |  3.39753282e-02 |   2.204 |  4.70016067e-03 |   2.092
   320 |  8.45838787e-03 |   2.006 |  1.14859291e-03 |   2.033
--------------------------------------------------------------
